In [29]:
# for 2 new models: Webcocket & time which look no pip install required
import os
import alpaca_trade_api as tradeapi
# we import 'ast' module to format reatime streaming message from Alpaca to the way in which our bot will understand it
import ast
import time
from datetime import datetime, timedelta
import json
import pandas as pd
import numpy as np
import numpy.random as rnd
from dotenv import load_dotenv
import requests
from pathlib import Path
import hvplot.pandas
import sqlalchemy
# Tablib stands for 'technical analysis library' used to analyse trading signal from candel charts
# to install ta-lib, type in terminal "conda install -c conda-forge ta-lib" 
import talib as ta
# Websocket will be used to connect with alpaca to extract real time stock price (candel stick chart)
# to install websocket: pip install websocket-client
import websocket
# import REST to extract historical stock price later
from alpaca_trade_api import REST, TimeFrame
# "Plotly" provides online graphing, analytics, and statistics tools for individuals and collaboration,
# in order for plotly to show graph in jupyter lab, you need to run in termimal: "jupyter labextension install jupyterlab-plotly"
# to run above command, you may also need to install nodeJS 12.0 + by use install command: "conda install nodejs -c conda-forge --repodata-fn=repodata.json"
import plotly.graph_objects as go
import plotly.express as px

In [9]:
# Load Alpaca info from ENV file
load_dotenv('api.env')
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')
type(alpaca_secret_key)

str

In [33]:
# Define variables which will be obained from FIRE to be inserted by user
stock_ticker = 'TSLA'


In [34]:
# Define "api" ojbect to be used later
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, "https://paper-api.alpaca.markets", "v2")

In [53]:
# Part ONE: Generating Trading signal 
## if the realtime prce is higher then certain percentage from the close price from 5 days ago, activate the trading bot
## Step 1: extract stock close price from 5 days agao

In [35]:
# use timedelta from Datetime to generate earlier date so we can extract historical data from alpaca later
days_to_subtract = 5
today = (datetime.today()).strftime('%Y-%m-%d')
print(f'today is {today}')
earlier_date_to_compare = today = (datetime.today()-timedelta(days=days_to_subtract)).strftime('%Y-%m-%d')
print(f'earlier_date_to_compare is {earlier_date_to_compare}')


today is 2022-05-08
earlier_date_to_compare is 2022-05-03


In [51]:
rest_client = REST(alpaca_api_key, alpaca_secret_key)
bars_from_earlier_date = rest_client.get_bars(stock_ticker, TimeFrame.Day, earlier_date_to_compare, earlier_date_to_compare).df
display(bars_from_earlier_date)
price_from_earlier_date = bars_from_earlier_date.iloc[0]['close']
print(f'The price_from_earlier_date is {price_from_earlier_date}')


,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2022-05-03 04:00:00+00:00,902.68,924.08,888.59,909.25,21187584,605288,908.677578


The price_from_earlier_date is 909.25


In [61]:
# Define variables which will be obained from FIRE to be inserted by user
capital_commited = 1000
# Use round operation to calculate number of shares to trade
shares_to_trade = capital_commited//price_from_earlier_date
# Note below numbers are in percentage terms (for example 2 means 2%)
percentage_change_for_trading_signal = 2
percentage_change_for_profit_loss_taking = 0.2
price_to_start_trading_bot = price_from_earlier_date * (1+percentage_change_for_trading_signal/100)
print(f'capital_commited,shares_to_trade, percentage_change_for_trading_signal,percentage_change_for_profit_loss_taking seperatly are {capital_commited,shares_to_trade,percentage_change_for_trading_signal,percentage_change_for_profit_loss_taking}')
print(f'price_to_start_trading_bot is {price_to_start_trading_bot}')

capital_commited,shares_to_trade, percentage_change_for_trading_signal,percentage_change_for_profit_loss_taking seperatly are (1000, 1.0, 2, 0.2)
price_to_start_trading_bot is 927.4350000000001


1

In [ ]:
# Part One
## Step 2: extact realtime stock price from Alpaca streaming data

In [52]:
# Below iex link should work with free account per alpaca 
socket = "wss://stream.data.alpaca.markets/v2/iex"
#socket = "wss://stream.data.alpaca.markets/v2/sip"

In [ ]:
# on_open, on_message, on_close below are defined for websockect steaming data
# within on_message, conditional statement is made check whether trading signal will be triggered
def on_open(ws):
    print("opened")
    auth_data = {"action":"auth","key":alpaca_api_key,"secret":alpaca_secret_key}
    ws.send(json.dumps(auth_data))
    listen_message = {"action":"subscribe","bars":[stock_ticker]}
    ws.send(json.dumps(listen_message))


def on_message(ws, message):
    print("received a message")
    print(message)
    formatted_message = ast.literal_eval(message)
    last_time = formatted_message[0].get("t")
    last_close = formatted_message[0].get("c")
    print(f'infor from preivous minitue: time is {last_time}; open is {last_open};high is {last_high};low is {last_low};close is {last_close};volumne is {last_volumne}')
# Once the realtime price exceeds the signal price, start the trading bot
    if(last_close>price_to_start_trading_bot):
        trading_bot()
    else:
        
def on_close(ws):
    print("closed connection")
    

In [ ]:
ws = websocket.WebSocketApp(socket, on_open=on_open, on_message=on_message, on_close=on_close)
ws.run_forever()

In [6]:
# below are code of simple tradinb bot with simple buy market and sell traliing order 
# for x in range(200):
def trading_bot()
    while True:
        try:
            api.get_position(stock_ticker)
            time.sleep(11)
        except:
            api.submit_order(symbol=stock_ticker,qty=shares_to_trade,side='buy',type='market',time_in_force='gtc')
            time.sleep(10)
            api.submit_order(symbol=stock_ticker,qty=shares_to_trade,side='sell',type='trailing_stop',trail_percent=percentage_change_for_profit_loss_taking,time_in_force='gtc')
